In [4]:
%reload_ext autoreload
%autoreload 2
from MultimodalDataset.GasData import GasDataProcessor
from MultimodalDataset.MedicalData import MedicalDataProcessor
from MultimodalDataset.ClimateData import ClimateDataProcessor
from src.utils import load_config, get_max_token_size

In [8]:
import yaml

file_path = "config/default/climate.yaml"
# Load the existing YAML file
with open(file_path, 'r') as file:
    config = yaml.safe_load(file)

# Iterate over input_window and output_window from 1 to 7
for i in range(1, 8):
    # Update the values in the config
    config['dataset']['input_window'] = i
    config['dataset']['output_window'] = i
    config['dataset']['hf_repo'] = f"Howard881010/climate-{i}day"
    
    # Save the updated configuration back to the original file
    with open(file_path, 'w') as outfile:
        yaml.dump(config, outfile, default_flow_style=False)
    
    print(f"Saved configuration with input_window={i} and output_window={i}.")
    climate_cfg = load_config('config/default/climate.yaml')

    processor = ClimateDataProcessor(climate_cfg['dataset'])
    climate_dataset = processor.get_dataset()
    max_length = get_max_token_size(climate_dataset)
    processor.push_to_huggingface(climate_dataset)
    print(max_length)

print("All configurations saved to gas.yaml.")

Saved configuration with input_window=1 and output_window=1.
Given the weather forecast of the first 1 day, predict the weather forecast of the next 1 day. Output the result **ONLY** in the following YAML format:
```
day_2_date:
day_2_weather_forecast:
```


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/707 [00:00<?, ?B/s]

3180
All configurations saved to gas.yaml.


In [2]:
from datasets import load_dataset, concatenate_datasets, DatasetDict

data1 = load_dataset('howard881010/climate-1day')
data2 = load_dataset('howard881010/climate-1day-mixed')
data3 = load_dataset('howard881010/climate-2day')
data4 = load_dataset('howard881010/climate-2day-mixed')
data5 = load_dataset('Howard881010/climate-3day')
data6 = load_dataset('howard881010/climate-3day-mixed')
data7 = load_dataset('Howard881010/climate-4day')
data8 = load_dataset('howard881010/climate-4day-mixed')
data9 = load_dataset('howard881010/climate-5day')
data10 = load_dataset('howard881010/climate-5day-mixed')
data11 = load_dataset('howard881010/climate-6day')
data12 = load_dataset('howard881010/climate-6day-mixed')
data13 = load_dataset('howard881010/climate-7day')
data14 = load_dataset('howard881010/climate-7day-mixed')

data_all = {}

for split in ['train', 'test', 'valid']:
    data_all[split] = concatenate_datasets([data1[split], data2[split], data3[split], data4[split], data5[split], data6[split], data7[split], data8[split], data9[split], data10[split], data11[split], data12[split], data13[split], data14[split]])

data_dict = DatasetDict({
    'train': data_all['train'],
    'test': data_all['test'],
    'valid': data_all['valid']
})

data_dict.push_to_hub('Howard881010/climate')
print(data1['test']['instruction'][0])

Generating train split:   0%|          | 0/750 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/94 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/94 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/748 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/94 [00:00<?, ? examples/s]

...

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/718 [00:00<?, ?B/s]

Given the weather forecast of the first 1 day, predict the weather forecast of the next 1 day.
Output the result **ONLY** in the following YAML format:
```
day_2_date:
day_2_weather_forecast:
```


In [12]:
import os
from evaluator.gpt_evaluator import ClimateProcessor
import pandas as pd
# climate_processor = ClimateProcessor()

# results_dir = "/home/ubuntu/multimodal/Data"
# results = pd.read_csv("/home/ubuntu/multimodal/Data/weather_raw.csv")

# jsonl_path = os.path.join(results_dir, "batch.jsonl")
# output_path = os.path.join(results_dir, "evaluator.txt")

# batch_object_id = climate_processor.create_and_run_batch_job(results, jsonl_path, input_text_column="Text")
# # if ready, outputs will be returned
# outputs = climate_processor.check_status_and_parse(batch_object_id, output_path)

# # # if you have the results saved already, just call
# outputs = climate_processor.parse_output(output_path)

ModuleNotFoundError: No module named 'evaluator'

In [6]:
import json
import pandas as pd

# Step 1: Load the .jsonl File
jsonl_file = '/home/ubuntu/multimodal/Data/outputs.jsonl'
data = []

with open(jsonl_file, 'r') as f:
    for line in f:
        climate_json = json.loads(line)
        text = []
        flattend_text = []
        for k, texts in climate_json.items():
            if k in ["overall_summary", "temperature_trend"]:
                texts = [text for text in texts if text != '']
                flattend_text.append(' '.join(str(x) for x in texts))

        data.append(" ".join(flattend_text))


# Step 2: Load the Existing CSV File
csv_file = '/home/ubuntu/multimodal/Data/weather_raw.csv'
df = pd.read_csv(csv_file)

# Step 3: Convert JSONL Data to a New Column
# Assuming each entry in the JSONL corresponds to a new column
# Replace 'your_key' with the correct key from your JSONL data
new_column = data
df['text'] = new_column  # Replace 'new_column_name' with your desired column name

new_df = df[['name', 'datetime', 'temp', 'text']]
new_df = new_df.rename(columns={'datetime': 'date'})

# Step 4: Save the Updated DataFrame to CSV
new_df.to_csv('updated_data.csv', index=False)

print("New CSV file with the additional column has been created as 'updated_data.csv'.")




New CSV file with the additional column has been created as 'updated_data.csv'.
